In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn import linear_model
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
car_prices = pd.read_csv('CarPrice_Assignment.csv')
car_prices.drop('car_ID', axis=1,inplace=True)
car_prices.head()

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [2]:
CompanyName = car_prices['CarName'].apply(lambda x : x.split(' ')[0])
car_prices.insert(3,"CompanyName",CompanyName)
car_prices.CompanyName.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

In [3]:
car_prices = car_prices.replace(to_replace ="alfa-romero", value ="alfa-romeo") 
car_prices = car_prices.replace(to_replace ="maxda", value ="mazda") 
car_prices = car_prices.replace(to_replace ="porcshce", value ="porsche") 
car_prices = car_prices.replace(to_replace ="toyouta", value ="toyota") 
car_prices = car_prices.replace(to_replace ="vokswagen", value ="volkswagen") 
car_prices = car_prices.replace(to_replace ="vw", value ="volkswagen")
car_prices = car_prices.replace(to_replace ="Nissan", value ="nissan")
car_prices.CompanyName.unique()

array(['alfa-romeo', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

In [4]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
car_prices['CompanyName']= label_encoder.fit_transform(car_prices['CompanyName'])
car_prices['carbody']= label_encoder.fit_transform(car_prices['carbody'])
car_prices['fueltype']= label_encoder.fit_transform(car_prices['fueltype'])
car_prices['aspiration']= label_encoder.fit_transform(car_prices['aspiration'])
car_prices["doornumber"] = car_prices.doornumber.replace({'two':2, "four":4})
car_prices['fuelsystem']= label_encoder.fit_transform(car_prices['fuelsystem'])
car_prices['enginelocation']= label_encoder.fit_transform(car_prices['enginelocation'])
car_prices['drivewheel']= label_encoder.fit_transform(car_prices['drivewheel'])
car_prices['enginetype']= label_encoder.fit_transform(car_prices['enginetype'])
car_prices['cylindernumber']= label_encoder.fit_transform(car_prices['cylindernumber'])

In [5]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [6]:
from sklearn.model_selection import train_test_split
Y = car_prices['price']
X = car_prices.drop(columns=['price','CarName'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.08, random_state = 0)

In [7]:
print("Shape of train set (X) :", X_train.shape)
print("Shape of train set (y) :", y_train.shape)
print("Shape of test set  (X) :", X_test.shape)
print("Shape of test set  (y) :", y_test.shape)

Shape of train set (X) : (188, 24)
Shape of train set (y) : (188,)
Shape of test set  (X) : (17, 24)
Shape of test set  (y) : (17,)


In [8]:
from tensorflow.keras.layers import Dense

input_number = X_train.shape[1] 

model = Sequential()

model.add(Dense(32, activation='relu', input_shape=(input_number,), name = "Hidden_Layer_1"))

In [9]:
model.add(Dense(8, activation='relu', name = "Hidden_Layer_2"))

In [10]:
model.add(Dense(2, activation='relu', name="Hidden_Layer_3"))

In [11]:
model.add(Dense(1, name = "Output"))

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden_Layer_1 (Dense)      (None, 32)                800       
                                                                 
 Hidden_Layer_2 (Dense)      (None, 8)                 264       
                                                                 
 Hidden_Layer_3 (Dense)      (None, 2)                 18        
                                                                 
 Output (Dense)              (None, 1)                 3         
                                                                 
Total params: 1,085
Trainable params: 1,085
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.fit(X_train, y_train, epochs=20, verbose=0)

In [14]:
mae_train = model.evaluate(X_train, y_train, verbose = 0)[1]
mae_test = model.evaluate(X_test, y_test, verbose = 0)[1]

print("\nMAE for train set : {:.3f}".format(mae_train))
print("MAE for test set    : {:.3f}".format(mae_test))


MAE for train set : 13354.725
MAE for test set    : 12412.527


In [15]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.fit(X_train, y_train, epochs=50, verbose=0)

In [16]:
mae_train = model.evaluate(X_train, y_train, verbose = 0)[1]
mae_test = model.evaluate(X_test, y_test, verbose = 0)[1]

print("\nMAE for train set : {:.3f}".format(mae_train))
print("MAE for test set    : {:.3f}".format(mae_test))


MAE for train set : 13354.426
MAE for test set    : 12412.229


In [17]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.fit(X_train, y_train, epochs=5000, verbose=0)

In [18]:
mae_train = model.evaluate(X_train, y_train, verbose = 0)[1]
mae_test = model.evaluate(X_test, y_test, verbose = 0)[1]

print("\nMAE for train set : {:.3f}".format(mae_train))
print("MAE for test set    : {:.3f}".format(mae_test))


MAE for train set : 13324.574
MAE for test set    : 12382.378


**Sonuçlar çok fazla değişiklik göstermedi.**